### 1 React Agent con Router y Especialista

In [13]:
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool 
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display


@tool
def multiply(a: float, b: float) -> float:
    """Multiplies two numbers."""
    result = a * b
    return f"El resultado es: {a * b} = {result}"

@tool
def sum(a: float, b: float) -> float:
    """Sums two numbers."""
    result = a + b
    return f"El resultado es: {a} + {b} = {result}"

@tool 
def web_search(quer:str) -> str:
    """Realiza una búsqueda en la web y devuelve un resumen de los resultados."""
    simulated_results = {
        "python programming": "Python es un lenguaje de programación interpretado, de alto nivel y de propósito general.",
        "langchain": "LangChain es un marco para desarrollar aplicaciones impulsadas por modelos de lenguaje.",
        "openai": "OpenAI es una organización de investigación en inteligencia artificial que desarrolla y promueve IA amigable."
    }
    for key in simulated_results:
        if key in quer.lower():
            return simulated_results[key]
    return "No se encontraron resultados relevantes."

@tool
def calculator(expression: str) -> str:
    """Evalue expresiones matematicas complejas."""
    try:
        result = eval(expression)
        return f"El resultado de la expresión '{expression}' es: {result}"
    except Exception as e:
        return f"Error al evaluar la expresión: {e}"


TOOLS = [multiply, sum, web_search, calculator]

#configuration LLM and tools 
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(TOOLS, parallel_tool_calls=False)

#Router inteligente
def router_by_task(state: MessagesState) -> Literal["math_specialist", "research_specialist", "general", "end"]:
    """Enruta segun el tipo de tarea detectada"""
    last_message = state["messages"][-1].content.lower()
    print("=== router_by_task")
    print(f"last_message: {last_message}")

    # detectar matematicas
    math_keywords = ["multiplica", "suma", "calcula", "resultado", "operación", "×", "+", "-", "*"]
    if any(keyword in last_message for keyword in math_keywords):
        return "math_specialist"

    # Detectar investigación
    research_keywords = ["busca", "investiga", "información", "qué es", "explica", "search"]
    if any(keyword in last_message for keyword in research_keywords):
        return "research_specialist"

    # Verificar si hay tool_calls pendientes
    last_ai_message = next((msg for msg in reversed(state["messages"]) if hasattr(msg, "tool_calls")), None)
    if last_ai_message and getattr(last_ai_message, "tool_calls", []):
        return "general"

    return "end"
   

def math_specialist(state: MessagesState):
 """ Especialista en matemtaicas y calculos """
 system_msg = SystemMessage(content="""Eres un matemático experto. 
    - Analiza el problema paso a paso
    - Usa las herramientas disponibles (multiply, sum_numbers, calculator)
    - Explica tu razonamiento antes de calcular
    - Verifica los resultados
    """)
 response = llm_with_tools.invoke([system_msg] + state["messages"])
 return {"messages": [response]} 

def research_specialist(state: MessagesState):
    """Especialista en investigación y búsquedas"""
    system_msg = SystemMessage(content="""Eres un investigador experto.
    - Usa web_search para encontrar información
    - Resume los hallazgos de forma clara y concisa
    - Cita las fuentes cuando sea relevante
    - Proporciona contexto útil
    """)
    response = llm_with_tools.invoke([system_msg] + state["messages"])
    return {"messages": [response]}

def general_assistant(state: MessagesState):
    """Asistente general para otras consultas"""
    system_msg = SystemMessage(content="""Eres un asistente útil y versátil.
    - Responde de forma clara y directa
    - Usa herramientas cuando sea necesario
    - Mantén un tono amigable y profesional
    """)
    response = llm_with_tools.invoke([system_msg] + state["messages"])
    return {"messages": [response]}

#contruimos el grafo 
graph = StateGraph(MessagesState)
toolNode = ToolNode(TOOLS)

#agregar nodos 
# no entiend lo que sigue ? graph.add_node("router", lambda state: {"messages": state["messages"]})

graph.add_node("router", lambda state: {"messages": state["messages"]})

graph.add_node("math_specialist", math_specialist)
graph.add_node("research_specialist",research_specialist)
graph.add_node("general",general_assistant)
graph.add_node("tools",toolNode)

#configura edges 
graph.add_edge(START,"router")
# 🔄 CONDITIONAL EDGE: Decisión inteligente basada en el contenido
graph.add_conditional_edges(
    "router",           # 1️⃣ Nodo origen: desde donde se toma la decisión
    router_by_task,     # 2️⃣ Función decisora: analiza el state y retorna una string
    {
        # 3️⃣ Mapeo: salida_de_función → nodo_destino
        # KEY (lo que retorna router_by_task) → VALUE (nodo al que ir)
        
        "math_specialist": "math_specialist",        # Si retorna "math_specialist" → va al nodo "math_specialist"
        "research_specialist": "research_specialist", # Si retorna "research_specialist" → va al nodo "research_specialist"  
        "general": "general",                        # Si retorna "general" → va al nodo "general"
        "end": "__end__"                            # Si retorna "end" → termina el grafo (nodo especial END)
    }
)
# cada especialista puede llamar a las tools 
graph.add_conditional_edges("math_specialist", tools_condition)
graph.add_conditional_edges("research_specialist", tools_condition)
graph.add_conditional_edges("general", tools_condition)

#despues de usar tools vovler al router 
graph.add_edge("tools","router")

#compilar
advanced_router_agent = graph.compile()

#display(Image(advanced_router_agent.get_graph(xray=True).draw_mermaid_png()))



In [12]:
result1 = advanced_router_agent.invoke({
        "messages": [HumanMessage(content="Multiplica 25 por 8 y luego suma 100 al resultado")]
    })
for msg in result1["messages"]:
        msg.pretty_print()

================================ Human Message =================================

Multiplica 25 por 8 y luego suma 100 al resultado
================================== Ai Message ==================================
Tool Calls:
  multiply (call_8pdkoYiKGOLxNPB1iYIwNhbj)
 Call ID: call_8pdkoYiKGOLxNPB1iYIwNhbj
  Args:
    a: 25
    b: 8
================================= Tool Message =================================
Name: multiply

El resultado es: 200.0 = 200.0
================================== Ai Message ==================================
Tool Calls:
  sum (call_2NBvmuRwbV9kOn2nhdmzCztA)
 Call ID: call_2NBvmuRwbV9kOn2nhdmzCztA
  Args:
    a: 200
    b: 100
================================= Tool Message =================================
Name: sum

El resultado es: 200.0 + 100.0 = 300.0
================================== Ai Message ==================================

El resultado de multiplicar 25 por 8 es 200. Luego, al sumarle 100 al resultado obtenemos 300.


In [14]:
result2 = advanced_router_agent.invoke({
        "messages": [HumanMessage(content="Busca información sobre LangGraph y explícame qué es")]
})
for msg in result2["messages"]:
        msg.pretty_print()

=== router_by_task
last_message: busca información sobre langgraph y explícame qué es
=== router_by_task
last_message: no se encontraron resultados relevantes.
================================ Human Message =================================

Busca información sobre LangGraph y explícame qué es
================================== Ai Message ==================================
Tool Calls:
  web_search (call_NoHH9nB8eDx0tDQyJvn4LlXD)
 Call ID: call_NoHH9nB8eDx0tDQyJvn4LlXD
  Args:
    quer: LangGraph
================================= Tool Message =================================
Name: web_search

No se encontraron resultados relevantes.
================================== Ai Message ==================================

No pude encontrar información específica sobre LangGraph en la web. Sin embargo, basándome en mi conocimiento matemático, puedo decirte que "LangGraph" podría referirse a un grafo que representa relaciones entre lenguajes de programación. Los grafos son estructuras matemáticas

In [15]:
result2 = advanced_router_agent.invoke({
        "messages": [HumanMessage(content="Busca información sobre langchain y explícame qué es")]
})
for msg in result2["messages"]:
        msg.pretty_print()

=== router_by_task
last_message: busca información sobre langchain y explícame qué es
=== router_by_task
last_message: langchain es un marco para desarrollar aplicaciones impulsadas por modelos de lenguaje.
================================ Human Message =================================

Busca información sobre langchain y explícame qué es
================================== Ai Message ==================================
Tool Calls:
  web_search (call_TaSfNS9X5YIKbra9o862MP4g)
 Call ID: call_TaSfNS9X5YIKbra9o862MP4g
  Args:
    quer: langchain
================================= Tool Message =================================
Name: web_search

LangChain es un marco para desarrollar aplicaciones impulsadas por modelos de lenguaje.
================================== Ai Message ==================================

LangChain es un marco para desarrollar aplicaciones impulsadas por modelos de lenguaje. Es una herramienta que facilita la creación de aplicaciones basadas en modelos de lenguaje.
